In [ ]:
import torch
from transformers import XLMRobertaTokenizer, XLMRobertaForTokenClassification
import shap
from lime.lime_text import LimeTextExplainer

# Load the fine-tuned model and tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained('fine_tuned_ner_model')
model = XLMRobertaForTokenClassification.from_pretrained('fine_tuned_ner_model')
model.eval()  # Set model to evaluation mode

# Define a prediction function
def predict_entities(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=2)
    return predictions, logits

# Using SHAP for Interpretability
def explain_with_shap(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    explainer = shap.Explainer(model, tokenizer)
    shap_values = explainer(inputs)

    # Plot the SHAP values
    shap.summary_plot(shap_values, feature_names=tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))

# Using LIME for Interpretability
def lime_predict(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    return torch.nn.functional.softmax(logits, dim=2).numpy()

def explain_with_lime(sentence):
    lime_explainer = LimeTextExplainer(class_names=["O", "B-Product", "B-PRICE", "B-LOC"])
    explanation = lime_explainer.explain_instance(sentence, lime_predict, num_features=10)
    
    # Show the LIME explanation
    explanation.show_in_notebook(text=True)

# Example Sentences for Explanation
sentences_to_explain = [
    "የንግድ ቤት ዋጋ ወቅታዊ ነው", 
    "ወርቅ እቃዎች የሚሸጡ ናቸው",  
]

# Explain predictions for example sentences
for sentence in sentences_to_explain:
    predictions, logits = predict_entities(sentence)
    print(f"Sentence: {sentence}")
    print(f"Predictions: {predictions}")
    explain_with_shap(sentence)
    explain_with_lime(sentence)
